In [2]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
print(f"PyTorch版本: {torch.__version__}")
print(f"GPU可用: {torch.cuda.is_available()}")
print(f"设备名称: {torch.cuda.get_device_name(0)}")


PyTorch版本: 2.9.0+cu126
GPU可用: True
设备名称: NVIDIA GeForce GTX 1650


In [3]:


# 定义数据预处理


# 加载数据集


# 创建DataLoader


10.2%


KeyboardInterrupt: 